# 05 Prepare data for machine learning
Clean and process data for machine learning

In [289]:
# Data manipulation
import pandas as pd
import numpy as np
import seaborn  as sns
import matplotlib.pyplot as plt
from time import gmtime, strftime
import ast
%matplotlib inline

In [307]:
#import pandas as pd
df = pd.read_csv('export_df_div_dropna.csv')

In [308]:
df = df.drop(['Unnamed: 0'], axis=1)
df.columns

Index(['name', 'bday', 'sex', 'age', 'age_diff', 'zodiac', 'zodiac_sp',
       'num_of_m', 'num_of_m_sp', 'num_of_child_cl', 'num_of_child_sp_cl',
       'num_of_role', 'num_of_role_sp', 'geohash', 'geo_distance', 'age_m_1',
       'age_div_1', 'age_m_sp_1', 'age_div_sp_1', 'divorce'],
      dtype='object')

## Preprocessing work

Convert sex column into numerical value

In [309]:
df['sex'] = df['sex'].map({'M':1 , 'F':0})
df['sex'].head()

0    1
1    1
2    1
3    1
4    1
Name: sex, dtype: int64

## Deal with missing data and prepare data for ML


In [311]:
# Check the missing value (%)
(df.isnull().sum()*100 / len(df)).sort_values(ascending=False)

num_of_child_sp_cl    78.284790
age_div_sp_1          73.203883
age_m_sp_1            73.203883
num_of_m_sp           71.553398
age_diff              68.964401
num_of_role_sp        68.834951
zodiac_sp             68.414239
geo_distance          67.443366
age_div_1             56.601942
num_of_child_cl       34.660194
num_of_role            4.142395
age_m_1                2.750809
bday                   2.394822
zodiac                 2.394822
age                    2.394822
geohash                0.906149
num_of_m               0.355987
divorce                0.000000
sex                    0.000000
name                   0.000000
dtype: float64

**1.** Deal with missing values in numerical columns

In [312]:
# numerical columns
df_num_vars = df[['age','age_diff', 'zodiac','zodiac_sp','num_of_child_cl',
                         'num_of_child_sp_cl','num_of_role','num_of_role_sp','geo_distance',
                         'age_m_1','age_m_sp_1','sex']];


In [313]:
# fill missing data with mean value
fill_mean = lambda col: col.fillna(col.mean())
df_num_vars = df_num_vars.apply(fill_mean, axis=0);
df_num_vars.shape

(3090, 12)

**2.** Deal with missing values in catagories column 

In [264]:
cat_df = df.select_dtypes(include=['object'])# Subset to a dataframe only holding the categorical columns

# Print how many categorical columns are in the dataframe - should be 147
cat_df.shape[1]

3

The categorical columns are 'name', 'geohash', and 'bday'. ''name ' is unnessesary for ML analysis. 'bday' is redundant to 'age' and 'zodiac' , so I removed these two columns. Next, do the get_dummies on the 'geohash' column,

In [314]:
#Create a copy of the dataframe
df_cat_vars =  df[['geohash']]

#Pull a list of the column names of the categorical variables
var = df_cat_vars.columns


# for each cat add dummy var, drop original column
df_cat_copy = pd.concat([df_cat_vars.drop(var, axis=1), \
                    pd.get_dummies(df_cat_vars[var], prefix=var, prefix_sep='_', drop_first=True,  dummy_na=True)], axis=1)
 

However, after running the feature importance of the randon forrest model, the geohash is not a very important features. So 'geohash' is not included into the dataset

Combind feature and label column together

In [330]:
df_Xy = pd.concat([df_num_vars, df['divorce']], axis=1)

In [331]:
df_Xy.shape

(3090, 13)

Seperate data to features and labels

In [332]:
df_X = df_Xy.drop(['divorce'], axis =1)
df_y = df_Xy[['divorce']]

Seperate data for male and female

In [337]:
#Male
df_Xy_M = df_Xy[df_Xy['sex'] ==1]
df_X_M = df_Xy_M.drop(['divorce', 'sex'], axis =1)
df_y_M = df_Xy_M[['divorce']]

#Female
df_Xy_F = df_Xy[df_Xy['sex'] ==0]
df_X_F = df_Xy_F.drop(['divorce', 'sex'], axis =1)
df_y_F = df_Xy_F[['divorce']]

In [338]:
print(df_X.shape)
print(df_y.shape)

(3090, 12)
(3090, 1)


In [339]:
# save data for ML
df_X.to_csv('export_df_X_forML.csv', header=True)
df_y.to_csv('export_df_y_forML.csv', header=True)
df_X_M.to_csv('export_df_X_M_forML.csv', header=True)
df_y_M.to_csv('export_df_y_M_forML.csv', header=True)
df_X_F.to_csv('export_df_X_F_forML.csv', header=True)
df_y_F.to_csv('export_df_y_F_forML.csv', header=True)